In [248]:
#Importing packages
import math
import random
import datetime
import sklearn
from sklearn import linear_model
from sklearn import tree
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [12]:
#Loading the data
project_data = pd.read_excel("(filepath)\\project.xlsx")
nlp_data_1 = pd.read_excel("(filepath)\\project_output_part_1.xlsx")
nlp_data_2 = pd.read_excel("(filepath)\\project_output_part_2.xlsx")

ModuleNotFoundError: No module named 'scikit_learn'

In [101]:
#We have access to all our data. Next, we will join our sets together
#First, let us sort the project dataset by the project id.
project_data.sort_values(by = "id")
#Next, we will combine our two NLP datasets and then sort it
nlp_data_full = pd.concat((nlp_data_1, nlp_data_2), axis=0, ignore_index=True)
nlp_data_full.sort_values(by = "id")
#Checking
for i in range(len(project_data)):
    if (not nlp_data_full["id"][i] == project_data["id"][i]):
        print("Error found!" + i)
#Drop redundant columns and then combine
nlp_data_dropped = nlp_data_full.drop(["id", "objective"], axis=1)
full_data = pd.concat((nlp_data_dropped, project_data), axis=1)
full_data.to_csv("(filepath)\\project_data_combined.csv")

In [527]:
full_data = pd.read_csv("(filepath)\\project_data_combined.csv")
full_data = full_data.drop("Unnamed: 0", axis = 1)
for i in range(len(full_data)):
    if not isinstance(full_data["scientific_domain"][i], str):
        full_data["scientific_domain"][i] = "no domain"
    if not isinstance(full_data["problem_type"][i], str):
        full_data["problem_type"][i] = "no problem"
    if not isinstance(full_data["expected_impact"][i], str):
        full_data["expected_impact"][i] = "no impact"
    if not isinstance(full_data["fundingScheme"][i], str):
        full_data["fundingScheme"][i] = "unknown scheme"
    if math.isnan(full_data["sustainability"][i]):
        full_data["sustainability"][i] = 0

C:\Users\jfros\AppData\Local\Temp\ipykernel_29604\1691130188.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_data["scientific_domain"][i] = "no domain"
C:\Users\jfros\AppData\Local\Temp\ipykernel_29604\1691130188.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_data["problem_type"][i] = "no problem"
C:\Users\jfros\AppData\Local\Temp\ipykernel_29604\1691130188.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_dat

In [438]:
#Next, we will preprocess the data to make it easier to use
impact_categories = []
for i in range(len(full_data)):
    topics = full_data["expected_impact"][i].split(",")
    for topic in topics:
        new_topic = topic.replace("[", '')
        new_topic = new_topic.replace("]", '')
        new_topic = new_topic.replace("'", '')
        new_topic = new_topic.replace(" ", '')
        new_topic = new_topic.lower()
        if not new_topic in impact_categories:
            impact_categories.append(new_topic)
impact_categories.remove('sustainability')
print(len(impact_categories))

22


In [441]:
ohe = np.array([0] * (len(impact_categories) * len(full_data)))
new_ohe = ohe.reshape([len(full_data), len(impact_categories)])

15341
15341


In [442]:
for i in range(len(full_data)):
    j = 0
    topics = full_data["expected_impact"][i].split(",")
    new_topics = []
    for topic in topics:
        new_topic = topic.replace("[", '')
        new_topic = new_topic.replace("]", '')
        new_topic = new_topic.replace("'", '')
        new_topic = new_topic.replace(" ", '')
        new_topic = new_topic.lower()
        new_topics.append(new_topic)
    new_topics
    for j in range(len(impact_categories)):
        if impact_categories[j] in new_topics:
            new_ohe[i,j] = 1
        j += 1    

In [444]:
lengths = [0] * len(full_data)
for i in range(len(full_data)):
    start_date = full_data["startDate"][i]
    startdate = start_date.split('-')
    time1 = datetime.datetime(int(startdate[0]), int(startdate[1]), int(startdate[2]))
    end_date = full_data["endDate"][i]
    enddate = end_date.split('-')
    time2 = datetime.datetime(int(enddate[0]), int(enddate[1]), int(enddate[2]))
    length = time2 - time1
    length = str(length)[0:3]
    length = length.replace(" ", '')
    lengths[i] = int(length)

In [276]:
for i in range(len(full_data)):
    if not full_data["masterCall"][i] == full_data["subCall"][i]:
        print(i)

In [445]:
schemes = []
for i in range(len(full_data)):
    if not full_data["fundingScheme"][i] in schemes:
        schemes.append(full_data["fundingScheme"][i])
ohe_schemes = np.array([0] * (len(full_data) * len(schemes)))
ohe_schemes = ohe_schemes.reshape([len(full_data), len(schemes)])
for i in range(len(full_data)):
    j = 0
    for scheme in schemes:
        if scheme == full_data["fundingScheme"][i]:
            ohe_schemes[i, j] = 1
        j += 1

In [446]:
eu_funding = np.array([0] * len(full_data))
for i in range(len(full_data)):
    funding = full_data["ecMaxContribution"][i]
    funding = funding.replace(',', '.')
    eu_funding[i] = float(funding)

In [524]:
domains = []
for i in range(len(full_data)):
    if not i in [5057, 12290, 12684]:
        if isinstance(full_data["scientific_domain"][i], str) and not full_data["scientific_domain"][i].lower() in domains:
            domains.append(full_data["scientific_domain"][i].lower())
ohe_domains = np.array([0] * (len(full_data) * len(domains)))
ohe_domains = ohe_domains.reshape([len(full_data), len(domains)])
for i in range(len(full_data)):
    if not i in [5057, 12290, 12684]:
        j = 0
        for domain in domains:
            if isinstance(full_data["scientific_domain"][i], str) and domain == full_data["scientific_domain"][i].lower():
                ohe_domains[i, j] = 1
            j += 1

In [523]:
problems = []
for i in range(len(full_data)):
    if not full_data["problem_type"][i].lower() in problems:
         problems.append(full_data["problem_type"][i].lower())
ohe_problems = np.array([0] * (len(full_data) * len(problems)))
ohe_problems = ohe_problems.reshape([len(full_data), len(problems)])
for i in range(len(full_data)):
    j = 0
    for problem in problems:
        if problem == full_data["problem_type"][i].lower():
            ohe_problems[i, j] = 1
        j += 1

In [ ]:
for i in range(len(problems)):
    full_data.insert(len(full_data.columns), problems[i], ohe_problems[:,i])
for i in range(len(impact_categories)):
    full_data.insert(len(full_data.columns), impact_categories[i] + "_impact", new_ohe[:,i])
for i in range(len(schemes)):
    full_data.insert(len(full_data.columns), schemes[i], ohe_schemes[:,i])
full_data.insert(27, "Duration", lengths)
full_data.insert(0, "EU_Funding", eu_funding)
for i in range(len(domains)):
    full_data.insert(len(full_data.columns), domains[i] + "_domain", ohe_domains[:,i])

In [664]:
full_data.to_csv("(filepath)\\project_data_filled.csv")

In [580]:
regression_columns = ["EU_Funding", "sustainability"]
for i in range(27, len(full_data.columns)):
    regression_columns.append(full_data.columns[i])

In [707]:
regression_data = full_data[regression_columns]
regression_data

,EU_Funding,sustainability,fundamental research,Duration,applied research,translational research,no problem,confidential,societal_impact,economic_impact,...,musicology_domain,oceanography_domain,astronomy_domain,urban studies_domain,nuclear engineering_domain,literature_domain,material science_domain,cognitive science_domain,confidential_domain,urban planning and sustainability - interdisciplinary_domain
0,1499998,0.0,1,182,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,12085363,0.0,1,219,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1489128,0.0,1,182,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,9957560,0.0,1,219,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1500000,0.0,1,182,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15336,75000,0.0,0,214,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
15337,75000,1.0,0,365,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
15338,75000,0.0,0,274,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
15339,75000,0.0,0,274,1,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [537]:
random.seed(0)
shuffled_data = regression_data.sample(frac=1).reset_index(drop=True)
training_set = shuffled_data[0:12272]
test_set = shuffled_data[12273:]

In [539]:
training_predictors = training_set[training_set.columns[1:]]
training_response = training_set["EU_Funding"]
regression = linear_model.LinearRegression()
regression.fit(training_predictors, training_response)

LinearRegression()

In [559]:
test_predictors = test_set[test_set.columns[1:]]
predictions = regression.predict(test_predictors)
mse = 0
for i in range(len(test_set)):
    mse = mse + (test_set["EU_Funding"][i + 12273] - predictions[i])**2
mse = mse / len(test_set)
print(mse)

1.5328452724590515e+21


In [708]:
log_funding = np.array([0.0] * len(regression_data))
for i in range(len(regression_data)):
    log_funding[i] = math.log(regression_data["EU_Funding"][i])
regression_data.insert(1, "log_funding", log_funding)
random.seed(0)
shuffled_data = regression_data.sample(frac=1).reset_index(drop=True)
training_set = shuffled_data[0:12272]
test_set = shuffled_data[12273:]

In [632]:
training_predictors = training_set[training_set.columns[2:]]
training_response = training_set["log_funding"]
regression = linear_model.LinearRegression()
regression.fit(training_predictors, training_response)

LinearRegression()

In [776]:
test_predictors = test_set[test_set.columns[2:]]
predictions = regression.predict(test_predictors)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- fund_cats


In [733]:
print(predictions)
print(test_set["log_funding"])
np.delete(predictions, 36)
test_set.drop([36 + 12273])
mse = 0
for i in range(len(test_set)):
    print(i)
    if not i == 36: #Value causes overflow in the following line, we chose to remove it
        mse += abs(math.exp(predictions[i]) - test_set["EU_Funding"][i + 12273]) * abs(math.exp(predictions[i]) - test_set["EU_Funding"][i + 12273])
mse = mse / (len(test_set) - 1)
print(mse)

['High Cost' 'High Cost' 'High Cost' ... 'High Cost' 'High Cost'
 'High Cost']
12273    15.604960
12274    12.349194
12275    11.967955
12276    14.730229
12277    12.107092
           ...    
15336    13.152310
15337    14.732364
15338    14.692015
15339    16.223855
15340    12.015590
Name: log_funding, Length: 3068, dtype: float64
0


TypeError: must be real number, not str

In [706]:
funding_categories = []
for i in range(len(regression_data)):
    if regression_data["EU_Funding"][i] > 20000000:
        funding_categories.append("Massive Cost")
    elif regression_data["EU_Funding"][i] > 200000:
        funding_categories.append("High Cost")
    else:
        funding_categories.append("Low Cost")

0 0 0


In [709]:
regression_data.insert(2, "fund_cats", funding_categories)

In [712]:
random.seed(0)
shuffled_data = regression_data.sample(frac=1).reset_index(drop=True)
training_set = shuffled_data[0:12272]
test_set = shuffled_data[12273:]

In [713]:
training_pred = training_set[regression_columns[3:]]
training_resp = training_set["fund_cats"]
classifier = tree.DecisionTreeClassifier()
classifier = classifier.fit(training_pred, training_resp)

In [714]:
test_pred = test_set[regression_columns[3:]]
training_resp = training_set["fund_cats"]
class_predictions = classifier.predict(test_pred)

In [715]:
lc_preds = training_set[training_set["fund_cats"].isin(["Low Cost"])]
lc_preds = lc_preds[lc_preds.columns[3:]]
hc_preds = training_set[training_set["fund_cats"].isin(["High Cost"])]
hc_preds = hc_preds[hc_preds.columns[3:]]
mc_preds = training_set[training_set["fund_cats"].isin(["Massive Cost"])]
mc_preds = mc_preds[mc_preds.columns[3:]]
lc_resps = training_set[training_set["fund_cats"].isin(["Low Cost"])]
lc_resps = lc_resps[lc_resps.columns[1]]
hc_resps = training_set[training_set["fund_cats"].isin(["High Cost"])]
hc_resps = hc_resps[hc_resps.columns[1]]
mc_resps = training_set[training_set["fund_cats"].isin(["Massive Cost"])]
mc_resps = mc_resps[mc_resps.columns[1]]

In [716]:
lc_regression = linear_model.LinearRegression()
lc_regression.fit(lc_preds, lc_resps)
hc_regression = linear_model.LinearRegression()
hc_regression.fit(hc_preds, hc_resps)
mc_regression = linear_model.LinearRegression()
mc_regression.fit(mc_preds, mc_resps)

LinearRegression()

In [749]:
lc_test = test_set[class_predictions == "Low Cost"]
lc_test = lc_test[lc_test.columns[3:]]
hc_test = test_set[class_predictions == "High Cost"]
hc_test = hc_test[hc_test.columns[3:]]
mc_test = test_set[class_predictions == "Massive Cost"]
mc_test = mc_test[mc_test.columns[3:]]
lc_true = test_set[class_predictions == "Low Cost"]
lc_true = lc_true[lc_true.columns[1]]
hc_true = test_set[class_predictions == "High Cost"]
hc_true = hc_true[hc_true.columns[1]]
mc_true = test_set[class_predictions == "Massive Cost"]
mc_true = mc_true[mc_true.columns[1]]

In [750]:
lc_predictions = lc_regression.predict(lc_test)
hc_predictions = hc_regression.predict(hc_test)
mc_predictions = mc_regression.predict(mc_test)

In [751]:
lc_test = lc_test.reset_index()
hc_test = hc_test.reset_index()
mc_test = mc_test.reset_index()
lc_true = lc_true.reset_index()
hc_true = hc_true.reset_index()
mc_true = mc_true.reset_index()

In [774]:
mse = 0
lc_true["log_funding"][i]
skipped = 0
for i in range(len(lc_predictions)):
    if lc_predictions[i] < 100:
        mse += abs(math.exp(lc_predictions[i]) - math.exp(lc_true["log_funding"][i])) * abs(math.exp(lc_predictions[i]) - math.exp(lc_true["log_funding"][i]))
    else:
        skipped += 1
for i in range(len(hc_predictions)):
    if hc_predictions[i] < 100:
        mse += abs(math.exp(hc_predictions[i]) - math.exp(hc_true["log_funding"][i])) * abs(math.exp(hc_predictions[i]) - math.exp(hc_true["log_funding"][i]))
    else:
        skipped += 1
for i in range(len(mc_predictions)):
    if mc_predictions[i] < 100:
        mse += abs(math.exp(mc_predictions[i]) - math.exp(mc_true["log_funding"][i])) * abs(math.exp(mc_predictions[i]) - math.exp(mc_true["log_funding"][i]))
    else:
        skipped += 1
mse = mse / (len(test_set) - skipped)